In [29]:
import numpy as np
import mne
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from scipy.signal import welch

In [30]:
# Step 1: Load EEG Dataset
# -------------------------------
# Replace with your actual EEG file path
file_path = r"C:\Users\Bhawna Bharti\Desktop\FYP25-26\eeg-motor-movementimagery-dataset-1.0.0\files\S001\S001R01.edf"
raw = mne.io.read_raw_edf(file_path, preload=True)  # preload ensures data is in memory

# Pick EEG channels only
raw.pick_types(eeg=True)

Extracting EDF parameters from C:\Users\Bhawna Bharti\Desktop\FYP25-26\eeg-motor-movementimagery-dataset-1.0.0\files\S001\S001R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


<RawEDF | S001R01.edf, 64 x 9760 (61.0 s), ~4.8 MiB, data loaded>

In [31]:
# -------------------------------
# Step 2: Preprocessing
# -------------------------------
# Bandpass filter 0.5–40 Hz
raw.filter(0.5, 40.0)

# Sampling frequency
sfreq = int(raw.info['sfreq'])

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1057 samples (6.606 s)



In [32]:
# Step 3: Labels
# -------------------------------
# Replace this with real dataset annotations
# Example: 0 = Left hand, 1 = Right hand
# Here we create dummy labels for demonstration (one per epoch)
epoch_length = sfreq  # 1 second epochs
n_epochs = (raw.n_times // epoch_length)
labels = np.random.randint(0, 2, size=n_epochs)


In [33]:
# Step 4: Feature Extraction (PSD)
# -------------------------------
def extract_features(signal, sfreq=256):
    """Extract PSD features (alpha, beta, gamma) from 1D signal."""
    freqs, psd = welch(signal, fs=sfreq, nperseg=sfreq*2)
    alpha = np.mean(psd[(freqs >= 8) & (freqs <= 13)])
    beta = np.mean(psd[(freqs >= 13) & (freqs <= 30)])
    gamma = np.mean(psd[(freqs >= 30) & (freqs <= 40)])
    return [alpha, beta, gamma]

# Segment data into epochs
data = raw.get_data()
X = []
y = []

for i in range(n_epochs):
    start = i * epoch_length
    end = start + epoch_length
    epoch = data[:, start:end]
    features = []
    for ch in epoch:
        features.extend(extract_features(ch, sfreq))
    X.append(features)
    y.append(labels[i])

X = np.array(X)
y = np.array(y)

D:\Anaconda\Lib\site-packages\scipy\signal\_spectral_py.py:600: UserWarning: nperseg = 320 is greater than input length  = 160, using nperseg = 160
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,


In [34]:
# Step 5: Train ML Model
# -------------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = SVC(kernel='rbf')
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.5384615384615384


In [35]:
# Step 6: Real-Time Simulation
# -------------------------------
print("\n--- Real-Time Simulation ---")
for i in range(len(X_test)):
    pred = clf.predict([X_test[i]])[0]
    action = "LEFT" if pred == 0 else "RIGHT"
    print(f"Epoch {i+1}: Prediction = {action}")


--- Real-Time Simulation ---
Epoch 1: Prediction = LEFT
Epoch 2: Prediction = RIGHT
Epoch 3: Prediction = LEFT
Epoch 4: Prediction = LEFT
Epoch 5: Prediction = RIGHT
Epoch 6: Prediction = LEFT
Epoch 7: Prediction = RIGHT
Epoch 8: Prediction = LEFT
Epoch 9: Prediction = LEFT
Epoch 10: Prediction = LEFT
Epoch 11: Prediction = LEFT
Epoch 12: Prediction = LEFT
Epoch 13: Prediction = RIGHT
